In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
os.environ["GDAL_DATA"] = "/srv/conda/envs/notebook/share/gdal" # need to specify to make gdal work
os.environ["PROJ_LIB"] = "/srv/conda/envs/notebook/share/proj" # need to specify to make pyproj work
os.environ["PROJ_DATA"] = "/srv/conda/envs/notebook/share/proj" # need to specify to make pyproj work
import numpy as np
import pandas as pd
from matplotlib import cm 
import matplotlib
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
import geopandas as gpd
from ipyleaflet import Map, basemaps, Polygon, Polyline, GeoData, LayersControl
from datetime import datetime
from dateutil.relativedelta import relativedelta
from rasterio import warp
import icepyx as ipx
import shutil

try:
    from ed.edcreds import getedcreds
except:
    print('No earthdata credentials found.')
    print('To set up, rename the folder "ed_example" to "ed".')
    print('Then, add your earthdata credentials to "ed/edcreds.py".')

from utils.nsidc import download_is2
from utils.readers import read_atl06
from utils.S2 import plotS2cloudfree, add_inset


## Show area of interest on a map

In [ ]:
# I just drew these randomly on a map...
shape = 'data/shapefiles/gilac-grounding.geojson'
shape_buffer = 'data/shapefiles/kreitzer-region-large.geojson'

gdf = gpd.read_file(shape)
geom = [i for i in gdf.geometry]
lons,lats = geom[0].exterior.coords.xy
shape_coords = list(zip(lats,lons))
shape_bounds = [[np.min(lats),np.min(lons)], [np.max(lats),np.max(lons)]]

gdf = gpd.read_file(shape_buffer)
geom = [i for i in gdf.geometry]
lons,lats = geom[0].exterior.coords.xy
shape_buffer_coords = list(zip(lats,lons))

m=Map(basemap=basemaps.Esri.WorldImagery,center=[np.mean(lats),np.mean(lons)],zoom=7)
m.fit_bounds(shape_bounds)
polygon = Polygon(locations=shape_buffer_coords, color="red", fill_color="red", name='polygon_buffered')
m.add_layer(polygon)
polygon2 = Polygon(locations=shape_coords, color="blue", fill_color="blue", name='polygon')
m.add_layer(polygon2)
m.add_control(LayersControl())
m

## Download the ATL06 ICESat-2 Data

In [ ]:
os.environ["EARTHDATA_USERNAME"] = getedcreds()[0]
os.environ["EARTHDATA_PASSWORD"] = getedcreds()[1]

short_name = 'ATL06'
spatial_extent = 'data/shapefiles/gilac-grounding.shp'
temporal = ['2018-01-01','2030-01-01']
region_a = ipx.Query(short_name, spatial_extent, temporal)
results = region_a.avail_granules(ids=True)
print('Found granules:')
results_print = np.array(results).flatten()
for i in range(np.min((len(results_print),10))):
    print(results_print[i])
if len(results_print)>10:
    print('...and %d more results.' % (len(results_print)-10))
region_a.avail_granules(ids=True)
output_dir = 'data/IS2/v006'

In [ ]:
# uncomment to download again
region_a.download_granules(output_dir)

### Get a list of all the ATL06 data files over this region

In [ ]:
search_for = 'ATL06_'
search_in = output_dir + '/'
filelist = [search_in+f for f in os.listdir(search_in) \
            if os.path.isfile(os.path.join(search_in, f)) & (search_for in f) & ('.h5' in f)]
filelist.sort()
print('There are %i files.' % len(filelist))

### Just pick the first file, and plot the ground tracks on the map

In [ ]:
file = filelist[0]
ancillary, dfs = read_atl06(file, verbose=False)
for gtx in dfs.keys():
    dfs[gtx] = dfs[gtx][dfs[gtx].qual_summary == 0]

### Some helpful ancillary data for an example ICESat-2 track

In [ ]:
ancillary

### an example dataframe with ATL06 data

In [ ]:
print(dfs.keys())

In [ ]:
dfs['gt3l']

### show a few tracks on a map to get an idea of coverage

In [ ]:
m=Map(basemap=basemaps.Esri.WorldImagery,
      center=list(np.mean(np.array(shape_coords), axis=0)),
      zoom=9)
m.fit_bounds(shape_bounds)

polygon = Polygon(locations=shape_coords, color="blue", fill_color="blue", name='polygon')
m.add_layer(polygon)

# plot the first 15 tracks
nfiles = 15
filelist.sort()
for file in filelist[:nfiles]:
    ancillary, dfs = read_atl06(file, verbose=False)
    for gtx in dfs.keys():
        dfs[gtx] = dfs[gtx][dfs[gtx].qual_summary == 0]
        coords = list(zip(dfs[gtx].lat, dfs[gtx].lon))
        weight = 2 if ancillary['gtx_strength_dict'][gtx] == 'strong' else 1
        name = 'track %d-%s' % (ancillary['rgt'], gtx)
        line = Polyline(locations=coords,color="red",weight=weight,fill=False, name=name, opacity=0.25)
        m.add_layer(line)

m.add_control(LayersControl())
m

## get a dataframe with basic file info

In [ ]:
df_files = pd.DataFrame({'filename': filelist})
df_files['granule_id'] = df_files.apply(lambda x: x.filename[x.filename.find('ATL06_'):], axis=1)
df_files['date'] = df_files.apply(lambda x: x.granule_id[6:10]+'-'+x.granule_id[10:12]+'-'+x.granule_id[12:14], axis=1)
df_files['track'] = df_files.apply(lambda x: int(x.granule_id[21:25]), axis=1)
df_files.head()

## pick a track and GTX and show some repeat tracks

In [ ]:
track = 721
gtx = 'gt3l'
# track = 226
# gtx = 'gt3l'
df_track = df_files[df_files.track == track].sort_values(by='date').reset_index(drop=True)
nfiles = len(df_track)
df_track.head()

### show the tracks to make sure there are no outliers

In [ ]:
m=Map(basemap=basemaps.Esri.WorldImagery,
      center=list(np.mean(np.array(shape_coords), axis=0)),
      zoom=9)
m.fit_bounds(shape_bounds)

polygon = Polygon(locations=shape_coords, color="blue", fill_color="blue", name='polygon')
m.add_layer(polygon)

for i in range(nfiles):
    filedata = df_track.iloc[i]
    ancillary, dfs = read_atl06(filedata.filename, verbose=False)
    if gtx in dfs.keys():
        df = dfs[gtx]
        df = df[df.qual_summary == 0].copy()
        coords = list(zip(df.lat, df.lon))
        weight = 2 if ancillary['gtx_strength_dict'][gtx] == 'strong' else 1
        name = '%s, track %d-%s' % (ancillary['date'], ancillary['rgt'], gtx)
        line = Polyline(locations=coords,color="red",weight=weight,fill=False, name=name, opacity=0.1)
        m.add_layer(line)

m.add_control(LayersControl())
m

### plot elevations

In [ ]:
df_repeat = df_track.copy()
nfiles = len(df_repeat)

fig, ax = plt.subplots(figsize=[9, 5], dpi=100)
handles = []
dfsgtx = []

colors = cm.cubehelix_r(np.linspace(0.05,0.9,nfiles))
for i in range(nfiles):
    filedata = df_repeat.iloc[i]
    ancillary, dfs = read_atl06(filedata.filename, verbose=False)
    if gtx in dfs.keys():
        df = dfs[gtx]
        df.loc[df.qual_summary != 0,'h'] = np.nan
        df['h_geoid_corrected'] = df.h - df.geoid_h
        df.loc[np.abs(df.h_geoid_corrected) > 1e4,'h_geoid_corrected'] = np.nan
        line, = ax.plot(df.xatc, df.h_geoid_corrected,lw=1, color=tuple(colors[i,:]), label=ancillary['date'])
        # scatt = ax.scatter(0, 0 , s=10, alpha=1, color=tuple(colors[i,:]), label=ancillary['date'])
        df['date'] = ancillary['date']
        handles.append(line)
        dfsgtx.append(df)
plt.legend(handles=handles, bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0, fontsize=10, ncols=1)
dfs_all = pd.concat(dfsgtx)
# ax.set_xlim((np.nanmin(dfs_all.lat), np.nanmax(dfs_all.lat)))
# ax.set_ylim((np.nanmin(dfs_all.h_geoid_corrected), np.nanmax(dfs_all.h_geoid_corrected)))
ax.set_xlabel('latitude')
ax.set_ylabel('elevation above geoid')
fig.tight_layout()

In [ ]:
region = 'Gillock / Kreitzer'
dfs_all.xatc -= dfs_all.xatc.min()
x_interp = np.arange(0, dfs_all.xatc.max(), 10)
dates = np.unique(dfs_all.date)
ndates = len(dates)
colors = cm.cubehelix_r(np.linspace(0.05,0.9,ndates))

# get the median across-track dist
yatcvals_interp = []
dhdy_interp = []
h_vals_interp = []
lat_interp = []
lon_interp = []
for i, date in enumerate(dates):
    thisdf = dfs_all[dfs_all.date == date]
    thisdf.loc[np.abs(thisdf.yatc)>10000, 'yatc'] = np.nan
    thisdf.loc[np.abs(thisdf.dh_fit_dy)>1, 'dh_fit_dy'] = np.nan
    yatcvals_interp.append(np.interp(x_interp, thisdf.xatc, thisdf.yatc))
    dhdy_interp.append(np.interp(x_interp, thisdf.xatc, thisdf.dh_fit_dy))
    h_vals_interp.append(np.interp(x_interp, thisdf.xatc, thisdf.h_geoid_corrected))
    lat_interp.append(np.interp(x_interp, thisdf.xatc, thisdf.lat))
    lon_interp.append(np.interp(x_interp, thisdf.xatc, thisdf.lon))

yatcvals_interp = np.array(yatcvals_interp)
dhdy_interp = np.array(dhdy_interp)
h_vals_interp = np.array(h_vals_interp)
lat_interp = np.array(lat_interp)
lon_interp = np.array(lon_interp)

yatcmid = np.array(yatcvals_interp)
yatc25 = np.nanpercentile(yatcmid,25, axis=0)
yatc75 = np.nanpercentile(yatcmid,75, axis=0)
yatcmid[(yatcmid <= yatc25) |  (yatcmid >= yatc75)] = np.nan
mean_yatc = np.array(pd.Series(np.nanmean(yatcmid,axis=0)).rolling(center=True,min_periods=1,window=1000).mean())#.reshape(1,-1)
offtracky = yatcvals_interp - mean_yatc
h_vals_slopecorr = h_vals_interp - offtracky * dhdy_interp
to_discard = np.abs(offtracky) > 20
h_vals_slopecorr[to_discard] = np.nan
lat_interp[to_discard] = np.nan
lon_interp[to_discard] = np.nan

hmid = h_vals_slopecorr.copy()
# h25 = np.nanpercentile(hmid, 25, axis=0)
# h75 = np.nanpercentile(hmid, 75, axis=0)
# hmid[(hmid <= h25) |  (hmid >= h75)] = np.nan
# np.sum(np.isnan(hmid),axis=0)
mean_h = np.nanmean(hmid, axis=0)

fig = plt.figure(figsize=[12, 4.5], dpi=80)
gs = fig.add_gridspec(ncols=5, nrows=1)
axs= [] 
axs.append(fig.add_subplot(gs[0, :2]))
axs.append(fig.add_subplot(gs[0, 2:]))
filename_base = ('%s_track%s_%s' % (region,track,gtx)).replace(' ','').replace('/','-')

ax = axs[1]
handles = []
for i, date in enumerate(dates):
    hplot = pd.Series(h_vals_slopecorr[i,:] - mean_h).rolling(center=True,min_periods=1,window=100).mean()
    if np.sum(np.isnan(hplot)) / len(hplot) < 0.1:
        line, = ax.plot(x_interp/1000, hplot, lw=1, color=tuple(colors[i,:]), label=date)
        handles.append(line)

meanline, = ax.plot(x_interp/1000, 0*mean_h, 'r--', lw=0.5, label='mean')
handles.append(meanline)

ax.legend(handles=handles, bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0, fontsize=10, ncols=1)
ax.set_ylabel('elevation above geoid')
ax.set_title(('%s (track %s-%s)' % (region, track, gtx)).upper())

_lat = lats
_xatc = x_interp/1000
def lat2xatc(l):
    return _xatc[0] + (l - _lat[0]) * (_xatc[1] - _xatc[0]) /(_lat[1] - _lat[0])
def xatc2lat(x):
    return _lat[0] + (x - _xatc[0]) * (_lat[1] - _lat[0]) / (_xatc[1] - _xatc[0])
secax = ax.secondary_xaxis(-0.075, functions=(xatc2lat, lat2xatc))
secax.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
secax.set_xlabel('along-track distance (km) / latitude', fontsize=10)
secax.tick_params(axis='both', which='major', labelsize=9)
secax.ticklabel_format(useOffset=False, style='plain')

ax = axs[0]
timeS2 = '2019-01-05T00:00:00Z'
imagery_filename = 'data/imagery/%s.tif'%filename_base
lons = np.nanmedian(lon_interp,axis=0)
lats = np.nanmedian(lat_interp,axis=0)
buff = np.max(x_interp)/2*1.1
myImage = plotS2cloudfree(lon=np.mean(lons), lat=np.mean(lats), date_time=timeS2, buffer_m=buff, max_cloud_prob=5, gamma_value=0.7, 
                imagery_filename=imagery_filename, ax=ax, download_imagery=True)
add_inset(ax, lat, lon, inset='antarctica', loc=[0.69, 0.01], width=0.3, height=0.25)
ximg, yimg = warp.transform(src_crs='epsg:4326', dst_crs=myImage.crs, xs=lons, ys=lats)
ax.annotate('', xy=(ximg[-1], yimg[-1]), xytext=(ximg[0], yimg[0]),
             arrowprops=dict(width=0.7, headwidth=5, headlength=5, color='r'),zorder=1000)
ax.plot(ximg, yimg, 'r-', lw=0.5, zorder=500)

fig.tight_layout()
fig.savefig('%s.jpg'%filename_base, dpi=300)